# `다항`선형회귀 - 보스턴 집값 예측하기
- `Polynomial` Regression
## 다항선형회귀 수행
### 1) 패키지 준비

In [ ]:
import sys
sys.path.append('../../')
import helper

from pandas import read_excel, DataFrame, merge
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

### 2) 데이터셋 준비

In [ ]:
origin = read_excel('https://data.hossam.kr/E04/boston.xlsx')
origin.info()

### 3) 데이터 전처리
- 결측치 확인
> 데이터에 결측치가 없으므로 생략

### 4) 탐색적 데이터 분석
- 명목형 처리

In [ ]:
origin['CHAS'] = origin['CHAS'].astype('category')
origin['CAT. MEDV'] = origin['CAT. MEDV'].astype('category')
origin.info()

- 기본통계 확인

In [ ]:
origin.describe()

- 상자그림

In [ ]:
plt.figure(figsize=(10, 5))
sb.boxplot(data=origin)
plt.show()
plt.close()

- 제외하기로 한 독립변수 제거
> 상관계수의 절대값이 0.7이상인 항목을 순차적으로 제거하여 회귀분석에 사용할 요인을 선정

In [ ]:
x = origin.drop(['TAX', 'DIS', 'NOX', 'INDUS'], axis=1)
x

- 명목형 변수만 추출

In [ ]:
categorical_df = x.filter(['CHAS', 'CAT. MEDV'])
categorical_df.info()

- 종속변수만 추출

In [ ]:
y = x['MEDV']
y

- 데이터프레임에서 추출한 필드를 제거

In [ ]:
x.drop(['CHAS', 'CAT. MEDV', 'MEDV'], axis=1, inplace=True)
x.info()

### 5) 데이터셋 분할
- 랜덤시드 고정

In [ ]:
np.random.seed(777)

- 데이터 표준화

In [ ]:
x_scaler = StandardScaler()
x_scale = x_scaler.fit_transform(x)
x_scale

- 표준화된 독립변수를 다항식으로 변환

In [ ]:
x_scale_df = DataFrame(x_scale, columns=x.columns)
x_scale_df

my_poly = PolynomialFeatures(degree = 2, include_bias = False)
my_fit = my_poly.fit_transform(x_scale_df)
x2 = DataFrame(my_fit, columns=my_poly.get_feature_names_out())
x2.head()

- 분리해 놓은 명목형 데이터와 결합

In [ ]:
x_df = merge(x2, caterogical_df, left_index=True, right_index=True)
x_df.info()

- 훈련 데이터와 검증 데이터로 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_df, 
                                                    y,
                                                    test_size=0.3,
                                                     random_state=777 )
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### 6) 모델 갸발
- 모델 정의

In [ ]:
my_model = Sequential()

# 1차원의 데이터를 입력으로 받고, 64개의 출력을 가지는 첫번째 Dense층
my_model.add(Dense(64, activation = 'relu', input_shape = (len(x_train.columns), )))
my_model.add(Dense(32, activation = 'relu'))

# 하나의 값을 출력 정답의 범위가 정해지지 않기 때문에 활성화 합수는 linear
my_model.add(Dense(1, activation = 'linear'))

my_model.compile(optimizer = 'adam'. loss='mse', metrics=['mae'])
my_model.summary()

### 7) 학습하기

In [ ]:
result = my_model.fit(x_train, 
                      y_train, 
                      epochs=500, 
                      validation_data=(x_test, y_test),
                      callbacks = [EarlyStopping(monitor='val_loss',
                                                 patience=10,
                                                 verbose=1),
                                   ReduceLROnPlateau(monitor='val_loss',
                                                     patience=3,
                                                     factor=0.5,
                                                     min_lr=0.001,
                                                     verbose=1)])

### 7) 학습 결과 평가

In [ ]:
helper.tf_result_plot(result)

evaluate1 = my_model.evaluate(x_train, y_train)
print('최종 훈련 손실률: %f, 최종 훈련 절대오차: %f' % (evaluate1[0], evaluate1[1]))

evaluate2 = my_model.evaluate(x_test, y_test)
print('최종 검증 손실률: %f, 최종 검증 절대오차: %f' % (evaluate2[0], evaluate2[1]))

### 8) 학습 결과 적용
- 테스트 데이터에 대한 예측 결과 산정

In [ ]:
train_pred = my_model.predict(x_train)
test_pred = my_model.predict(x_test)

- 결과 데이터셋 구성

In [ ]:
train_df = DataFrame(x_train, columns=x_train.columns)
train_df['관측치'] = y_train.values
train_df['예측치'] = train_pred.flatten()
train_df['훈련오차'] = train_df['관측치'] - train_df['예측치']
train_df.head()

test_df = DataFrame(x_test, columns=x_train.columns)
test_df['관측치'] = y_test.values
test_df['예측치'] = test_pred.flatten()
test_df['훈련오차'] = test_df['관측치']-test_df['예측치']
test_df.head()

- 실제 결과값과 머신러닝에 의한 예측값 비교

In [ ]:
for key in ['CRIM','ZN','RM','AGE','RAD','PTRATIO','B','LSTAT']:
    helper.regplot(train_df[key], train_df['관측치'], train_df['예측치'], "훈련데이터",
                   test_df[key], test_df['관측치'], test_df['예측치'], "검증데이터",
                   figsize=(15, 5))